In [1]:
# imports and calls
from IPython.display import clear_output
import pennylane as qml
import numpy as np
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.state_preparations import MottonenStatePreparation
import torch
import torch.nn as nn
import tensorflow as tf
from sklearn.decomposition import PCA
from keras.models import Model
from keras import layers, losses

import pennylane as qml
from pennylane import numpy as np
import autograd.numpy as anp
import matplotlib.pyplot as plt
global Embedding

clear_output()


Unitaries


In [3]:
# Different unitary ansatz to test from different papers (Hur el. at provides a comprehensive list of ansatzes in their code.)

# Unitary Ansatze for Convolutional Layer
def U_TTN(params, wires):  # 2 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_5(params, wires):  # 10 params
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRZ(params[4], wires=[wires[1], wires[0]])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_6(params, wires):  # 10 params
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])
    qml.RZ(params[2], wires=wires[0])
    qml.RZ(params[3], wires=wires[1])
    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])
    qml.RX(params[6], wires=wires[0])
    qml.RX(params[7], wires=wires[1])
    qml.RZ(params[8], wires=wires[0])
    qml.RZ(params[9], wires=wires[1])


def U_9(params, wires):  # 2 params
    qml.Hadamard(wires=wires[0])
    qml.Hadamard(wires=wires[1])
    qml.CZ(wires=[wires[0], wires[1]])
    qml.RX(params[0], wires=wires[0])
    qml.RX(params[1], wires=wires[1])


def U_13(params, wires):  # 6 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRZ(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRZ(params[5], wires=[wires[0], wires[1]])


def U_14(params, wires):  # 6 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CRX(params[2], wires=[wires[1], wires[0]])
    qml.RY(params[3], wires=wires[0])
    qml.RY(params[4], wires=wires[1])
    qml.CRX(params[5], wires=[wires[0], wires[1]])


def U_15(params, wires):  # 4 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])


def U_SO4(params, wires):  # 6 params
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0])
    qml.RY(params[5], wires=wires[1])


def U_SU4(params, wires): # 15 params
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])

    qml.CRX(params[4], wires=[wires[1], wires[0]])
    qml.CRX(params[5], wires=[wires[0], wires[1]])

# Pooling Layer

def Pooling_ansatz1(params, wires): #2 params
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

def Pooling_ansatz2(wires): #0 params
    qml.CRZ(wires=[wires[0], wires[1]])

def Pooling_ansatz3(*params, wires): #3 params
    qml.CRot(*params, wires=[wires[0], wires[1]])




#ancilla interactions layer J_OUR
def Ancilla_ansatz1(params, wires): #0 + 3*3 params
    qml.CNOT(wires=[wires[3],wires[0]])
    qml.CNOT(wires=[wires[0],wires[1]])
    qml.CNOT(wires=[wires[1],wires[2]])
    qml.CNOT(wires=[wires[2],wires[3]])
    qml.CNOT(wires=[wires[1],wires[4]])
    qml.CNOT(wires=[wires[2],wires[5]])
    qml.CNOT(wires=[wires[3],wires[6]])

    qml.Rot(params[0], params[1], params[2], wires=4)
    qml.Rot(params[3], params[4], params[5], wires=5)
    qml.Rot(params[6], params[7], params[8], wires=6)



def Ancilla_toffoli_ansantz(params, wires):
    qml.Toffoli(wires=[wires[1],wires[2],wires[3]])
    qml.Toffoli(wires=[wires[0],wires[1],wires[2]])

    qml.RX(params[0], wires[0])
    qml.RX(params[1], wires[1])
    qml.RX(params[2], wires[2])
    qml.RX(params[3], wires[3])

    qml.Toffoli(wires=[wires[0],wires[3],wires[1]])
    qml.RY(params[4], wires[0])
    qml.RY(params[5], wires[1])
    qml.RY(params[6], wires[2])
    qml.RY(params[7], wires[3])

    qml.Toffoli(wires=[wires[3],wires[2],wires[0]])
    
    qml.RZ(params[8], wires[0])
    qml.RZ(params[9], wires[1])
    qml.RZ(params[10], wires[2])
    qml.RZ(params[11], wires[3])

    


def Ancilla_toffoli_ansantz_paramless(wires):

    qml.Toffoli(wires=[wires[1],wires[2],wires[3]])
    qml.Toffoli(wires=[wires[0],wires[1],wires[2]])
    qml.Toffoli(wires=[wires[0],wires[3],wires[1]])
    qml.Toffoli(wires=[wires[3],wires[2],wires[0]])




Embedding

In [5]:
# the data embedding function

def data_embedding(X, embedding_type='Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires=range(8), normalize=True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires=range(8), rotation='Y')


Convolutional layers using unitaries

In [6]:
# Quantum Circuits for Convolutional layers
def conv_layer1(U, params):
    U(params, wires=[0, 7])
    for i in range(0, 8, 2):
        U(params, wires=[i, i + 1])
    for i in range(1, 7, 2):
        U(params, wires=[i, i + 1])
def conv_layer2(U, params):
    U(params, wires=[0, 6])
    U(params, wires=[0, 2])
    U(params, wires=[4, 6])
    U(params, wires=[2, 4])
def conv_layer3(U, params):
    U(params, wires=[0,4])

# Quantum Circuits for Pooling layers
def pooling_layer1(V, params):
    for i in range(0, 8, 2):
        V(params, wires=[i + 1, i])
def pooling_layer2(V, params):
    V(params, wires=[2,0])
    V(params, wires=[6,4])
def pooling_layer3(V, params):
    V(params, wires=[0,4])


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

    
def ancilla_interaction_layer(A, params):
    A(params, wires=[0, 2, 4, 6, 8, 9, 10])



def ancilla_toffoli_interaction_layer(A, params):
    A(params, wires=[0, 2, 4, 6])

def ancilla_toffoli_interaction_layer_paramless(A):
    A(wires=[0, 2, 4, 6])



def QCNN_our_structure(U, params, U_params): # number of weights is 15*2 + 2 + 8 = U_params, params =  weights
    
    param1 = params[0:U_params]
    param2 = params[U_params: 2 * U_params]
    param3 = params[2 * U_params: 3 * U_params]
    param4 = params[3 * U_params: 4 * U_params]
    param5 = params[4 * U_params: 4*U_params + 2]
    param6 = params[4*U_params + 2: 4*U_params + 11]         #total 71 params for usu4
    
    # Pooling Ansatz1 is used by default
    # increase the layers to 2
    conv_layer1(U, param1)
    conv_layer1(U, param2)
    pooling_layer1(Pooling_ansatz1, param5)
    conv_layer2(U, param3)
    conv_layer2(U, param4)
    ancilla_interaction_layer(Ancilla_ansatz1, param6)
    

def QCNN_our_structure_toffoli(U, params, U_params): # number of weights is 15*2 + 2 + 8 = U_params, params =  weights

    param1 = params[0:U_params]
    param4 = params[U_params: 2 * U_params]
    param5 = params[2 * U_params: 2*U_params + 2]
    param6 = params[2*U_params + 2: 2*U_params + 14]
    param7 = params[2*U_params + 14: 2*U_params + 23]


    conv_layer1(U, param1)
    conv_layer1(U, param1)
    #conv_layer1(U, param2)
    pooling_layer1(Pooling_ansatz1, param5)
    ancilla_toffoli_interaction_layer_paramless(Ancilla_toffoli_ansantz_paramless)
    #conv_layer2(U, param3)
    conv_layer2(U, param4)
    conv_layer2(U, param4)
    ancilla_toffoli_interaction_layer(Ancilla_toffoli_ansantz, param6)
    ancilla_interaction_layer(Ancilla_ansatz1, param7)

    

Making the circuit

In [7]:
dev = qml.device('default.qubit', wires = 11) # must be changed to 8 + number of classes that are to be used
@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type='Amplitude', cost_fn='cross_entropy'):

    # Data Embedding
    data_embedding(X, embedding_type=embedding_type)

    # Quantum Convolutional Neural Network
    if U == 'U_6':
        QCNN_our_structure_toffoli(U_6, params, U_params)
    elif U == 'U_SU4':
        QCNN_our_structure_toffoli(U_SU4, params, U_params)
    else:
        print("Invalid Unitary Ansatze")
        return False
   

    if cost_fn == 'cross_entropy':
        return qml.expval(qml.PauliZ(8)), qml.expval(qml.PauliZ(9)) , qml.expval(qml.PauliZ(10)) 
    else :
        print("Invalid Cost Function")
        return False



Cost and loss functions

In [8]:
# cost and loss functions, must be changed to one hot encoding

def cross_entropy(labels, predictions):  #J_OUR
    loss = 0
    for i, j in zip(labels, predictions):
        loss = loss + i*np.log(j)
        #print(loss)
    return -1*loss

def cost(params, X, Y, U, U_params, embedding_type, circuit, cost_fn): #J_OUR
    if circuit == 'QCNN':
        loss = 0
        for l , x in zip(Y, X):

            predictions = QCNN(x, params, U, U_params, embedding_type, cost_fn=cost_fn)
            predictions = softmax(predictions)
            loss = loss + cross_entropy(l, predictions)
        loss = loss/len(Y)
        
    else:
        print("Invalid Cost Function")
        return False
    return loss

Training params and training 

In [9]:
# Circuit training parameters
steps = 500
learning_rate = 0.01
batch_size = 60

def circuit_training(X_train, Y_train, X_test, Y_test, U, U_params, embedding_type, circuit, cost_fn, binary):
    if circuit == 'QCNN':
        total_params = 2*U_params + 23 #J_OUR

    params = np.random.randn(total_params, requires_grad=True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []

    f = open('Double_layer_same_weights_our_results_multi/Toffoli_multi_auto8_usu4_lr0.01_nonad_V4.txt', 'a')
    f.write("initial params: " + str(params))
    f.write("\n\n")
    f.close()

    for it in range(steps):
        f = open('Double_layer_same_weights_our_results_multi/Toffoli_multi_auto8_usu4_lr0.01_nonad_V4.txt', 'a')
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = X_train[batch_index]
        Y_batch = Y_train[batch_index]


        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type, circuit, cost_fn),
                                                     params)
        loss_history.append(cost_new)

        if it % 10 == 0:           
            correct = 0
            total = 0
            for l, x in zip(Y_test, X_test):
                act_label = np.argmax(l) # act_label = 1 (index)
                pred_label = np.argmax(softmax(QCNN(x, params, U, U_params, embedding_type, cost_fn))) # pred_label = 1 (index)
                if(act_label == pred_label):
                    correct += 1
                total += 1
            accuracy = (correct/total)
            print("iteration: ", it, " cost: ", cost_new, "accuracy: ", str(accuracy))
            f.write("iteration:" + str(it) + " cost:" + str(cost_new) + " "   " accuracy " + str(accuracy))
            f.write("\n")

            if accuracy > 0.850:
                f.write("params: " + str(params) + "\n\n")

           # if (it < 250) and (it % 50 ==0):
                 #opt.stepsize =  learning_rate/2 
                
        f.close()

    return loss_history, params

Data loads and processes, Prep Xtrain Xtest

In [10]:
# prepping Xtrain Ytrain, must be one hot coded 

def data_load_and_process(dataset, classes=[0, 1, 2], feature_reduction='resize256', binary=False):
    if dataset == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
        y_test
        #print('loaded fashion mnnist')

    elif dataset == 'mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    x_train, x_test = x_train[..., np.newaxis] / 255.0, x_test[..., np.newaxis] / 255.0  # normalize the data
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]) | (y_train == classes[2]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]) | (y_test == classes[2]))

    X_train, X_test = x_train[x_train_filter_01], x_test[x_test_filter_01]
    Y_train, Y_test = y_train[x_train_filter_01], y_test[x_test_filter_01]


    if binary == False:

        for i in range(len(Y_train)):
            if Y_train[i] == classes[0]:
                Y_train[i] = 0
            elif Y_train[i] == classes[1]:
                Y_train[i] = 1
            elif Y_train[i] == classes[2]:
                Y_train[i] = 2

        for i in range(len(Y_test)):
            if Y_test[i] == classes[0]:
                Y_test[i] = 0
            elif Y_test[i] == classes[1]:
                Y_test[i] = 1
            elif Y_test[i] == classes[2]:
                Y_test[i] = 2

    if binary == True:
        Y_train = [1 if y == classes[0] else -1 for y in Y_train]
        Y_test = [1 if y == classes[0] else -1 for y in Y_test]
        
    if feature_reduction == 'resize256':
        
        X_train = tf.image.resize(X_train[:], (256, 1)).numpy()
        X_test = tf.image.resize(X_test[:], (256, 1)).numpy()
        X_train, X_test = tf.squeeze(X_train).numpy(), tf.squeeze(X_test).numpy()

        Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=3)
        Y_test = tf.keras.utils.to_categorical(Y_test, num_classes=3)   

        return X_train, X_test, Y_train, Y_test

    elif feature_reduction == 'autoencoder8':
        
        latent_dim = 8



        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim
                self.encoder = tf.keras.Sequential([
                    layers.Flatten(),
                    layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                    layers.Dense(784, activation='sigmoid'),
                    layers.Reshape((28, 28))
                ])

            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded

        autoencoder = Autoencoder(latent_dim)

        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(X_train, X_train,
                        epochs=10,
                        shuffle=True,
                        validation_data=(X_test, X_test))

        X_train, X_test = autoencoder.encoder(X_train).numpy(), autoencoder.encoder(X_test).numpy()

        # Rescale for Angle Embedding
        
        if feature_reduction == 'autoencoder8':
            X_train, X_test = (X_train - X_train.min()) * (np.pi / (X_train.max() - X_train.min())), \
                              (X_test - X_test.min()) * (np.pi / (X_test.max() - X_test.min()))
                              
        Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=3)
        Y_test = tf.keras.utils.to_categorical(Y_test, num_classes=3)

        return X_train, X_test, Y_train, Y_test

Encoding to embedding

In [13]:
# converts encoding to embedding

def Encoding_to_Embedding(Encoding):
    #Amplitude Embedding / Angle Embedding
    print(type(Encoding))
    if Encoding == 'resize256':
        Embedding = 'Amplitude'
        #print("kano dhuktese na bhai")
    elif Encoding == 'autoencoder8':
        Embedding = 'Angle'
    else :
        print("Inavlid embedding")
    return Embedding


In [14]:
def main_function(dataset, classes, Unitaries, U_num_params, Encodings, circuit, cost_fn, binary=True):

    f = open('Result/result_US.txt', 'a')
    
    U = Unitaries
    U_params = U_num_params
    Encoding = Encodings
    Embedding = Encoding_to_Embedding(Encoding)

    feature_reduction = Encoding


    X_train, X_test, Y_train, Y_test = data_load_and_process(dataset, classes=classes,
                                                                feature_reduction=Encoding, binary=binary)

    print("\n")
    print("Loss History for " + circuit + " circuits, " + U + " " + Encoding + " with " + cost_fn)
    loss_history, trained_params = circuit_training(X_train, Y_train, X_test, Y_test, U, U_params, Embedding, circuit, cost_fn, binary) 
    
    
    f.write("Loss History for " + circuit + " circuits, " + U + " " + Encoding + " with " + cost_fn)
    f.write("\n")
    f.write(str(loss_history))
    f.write("\n")
    #f.write("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))
    #f.write("\n")
    f.write("\n")

    f.close()

    return loss_history, trained_params

In [17]:
Unitaries = 'U_SU4'
U_num_params = 15
Encodings = 'autoencoder8'
dataset = 'fashion_mnist'
classes = [0,1,2]
binary = False
cost_fn = 'cross_entropy'


loss_history, trained_params = main_function(dataset, classes, Unitaries, U_num_params, Encodings, circuit='QCNN', cost_fn=cost_fn, binary=binary)




<class 'str'>
Done with making three classes
Epoch 1/10
563/563 [==============================] - 2s 3ms/step - loss: 0.0514 - val_loss: 0.0351
Epoch 2/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0313 - val_loss: 0.0277
Epoch 3/10
563/563 [==============================] - 1s 1ms/step - loss: 0.0263 - val_loss: 0.0253
Epoch 4/10
563/563 [==============================] - 1s 1ms/step - loss: 0.0245 - val_loss: 0.0239
Epoch 5/10
563/563 [==============================] - 1s 1ms/step - loss: 0.0233 - val_loss: 0.0230
Epoch 6/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0225 - val_loss: 0.0222
Epoch 7/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0219 - val_loss: 0.0217
Epoch 8/10
563/563 [==============================] - 1s 1ms/step - loss: 0.0215 - val_loss: 0.0214
Epoch 9/10
563/563 [==============================] - 1s 2ms/step - loss: 0.0211 - val_loss: 0.0210
Epoch 10/10
563/563 [==============================] - 

C:\Users\masht\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\autograd\numpy\numpy_vjps.py:698: ComplexWarning: Casting complex values to real discards the imaginary part
  onp.add.at(A, idx, x)


iteration:  0  cost:  1.0830105814267594 accuracy:  0.439
iteration:  10  cost:  1.048385128953358 accuracy:  0.6576666666666666
iteration:  20  cost:  1.034815106559373 accuracy:  0.682
iteration:  30  cost:  1.0409826948108347 accuracy:  0.7333333333333333
iteration:  40  cost:  1.0245977745742036 accuracy:  0.7816666666666666
iteration:  50  cost:  1.0102833232566018 accuracy:  0.758
iteration:  60  cost:  1.012311465659457 accuracy:  0.8123333333333334
iteration:  70  cost:  1.0126321903127922 accuracy:  0.8643333333333333
iteration:  80  cost:  0.9932203791546854 accuracy:  0.868
iteration:  90  cost:  0.9953783243189869 accuracy:  0.87
iteration:  100  cost:  0.9753308696037962 accuracy:  0.867
iteration:  110  cost:  0.9803530841015089 accuracy:  0.8146666666666667
iteration:  120  cost:  0.9613783954638265 accuracy:  0.8376666666666667
iteration:  130  cost:  0.971213730187008 accuracy:  0.7856666666666666
iteration:  140  cost:  0.9643335197217947 accuracy:  0.7903333333333333

In [ ]:
print(trained_params)

In [ ]:
f = open('Double_layer_same_weights_our_results_multi/Toffoli_multi_auto8_usu4_lr0.01_nonad_V4.txt', 'a')
f.write("\n")
f.write("loss history: " + str(loss_history))
f.close()